# TensorFlow tutorial
_MILA, November 2017_

## Protip: browsing through the TensorFlow API

The [devdocs.io](http://devdocs.io/) website is an amazing resource to browse through the TensorFlow Python API (as well as many other APIs such as numpy or the Python API itself).

# Using TensorFlow at MILA

The most straightforward way to access TensorFlow using the MILA software stack is through the `tf1.4` conda environment. To activate the `tf1.4` environment, use the command

```bash
source activate tf1.4
```

To return back to normal, simply use the bash command

```bash
source deactivate
```

# Installing TensorFlow outside MILA

Follow the [online documentation](https://www.tensorflow.org/install/), which describes how to install TensorFlow for all major platforms (Linux, macOS, Windows) in various ways (`virtualenv`, native `pip`, Docker, Anaconda).

# Importing TensorFlow

TensorFlow is imported as a Python package using the following statement:

In [1]:
import tensorflow as tf

_Note: the `tensorflow` package is usually aliased to `tf` for convenience._

# Paradigm

TensorFlow separates the _definition_ of computation from its _execution_.

Computation is defined via a [_dataflow_](https://en.wikipedia.org/wiki/Dataflow_programming) _graph_, i.e., a graph where nodes represent units of computation and the edges represent the data consumed or produced by the computation.

TensorFlow calls these edges _tensors_ (not to be confused with the mathematical object of the same name). In TensorFlow parlance, a tensor is simply a multi-dimensional array of a certain data type.

# Constant, variable, placeholder, and random tensors

Many types of tensors may be used as input to the computation graph. We will cover four of them here: constant, variable, placeholder, and random tensors.

## Constant

A constant tensor always evaluates to the same value. It can be created using the `tf.constant` function:

In [2]:
c = tf.constant(value=42.0, name='c')
print(c)

Tensor("c:0", shape=(), dtype=float32)


The `name` argument is not strictly necessary, but it is considered good practice to name things in TensorFlow, as it facilitates visualizing the computation graph and debugging.

To get the value associated with a constant tensor, we evaluate it within a session:

In [3]:
with tf.Session() as session:
    print('The value for c is {}'.format(session.run(c)))

The value for c is 42.0


The value for a constant _always_ stays the same, be it within the same session or across different sessions:

In [4]:
with tf.Session() as session:
    print('The value for c is {}'.format(session.run(c)))
    print('The value for c is {}'.format(session.run(c)))

with tf.Session() as session:
    print('The value for c is {}'.format(session.run(c)))

The value for c is 42.0
The value for c is 42.0
The value for c is 42.0


## Variable

It can be useful for an input tensor's value to evolve across the lifetime of a session. For instance, a tensor's value can represent the weights of a neural network which we want to update using gradient descent.

Tensors with this property are called _variables_. The preferred way to create variables is via `tf.get_variable`:

In [5]:
v = tf.get_variable(
    name='v', shape=[2], dtype=tf.float32,
    initializer=tf.zeros_initializer())
print(v)

<tf.Variable 'v:0' shape=(2,) dtype=float32_ref>


This time, the `name` argument is required. This is because TensorFlow refers to variables by name. As such, TensorFlow expects the name for the variable to be unique. Trying to create a variable with the same name will result in an error:

In [6]:
# Throughout this tutorial, we will wrap statements that
# we know will cause an error to be raised with a try-except
# block to print the error message only, and not the whole
# stack trace.
try:
    tf.get_variable(name='v')
except ValueError as e:
    print(str(e).split('\n')[0])

Variable v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:


_(In case you are wondering, there is a way to bypass this behavior and retrieve by name a variable which has already been created. More on that later.)_

Note that a variable's value only makes sense _within the context of a session_. Furthermore, a variable's initial value has to be set before it can be used. See what happens if we try to evaluate `v` within a session:

In [7]:
with tf.Session() as session:
    try:
        session.run(v)
    except tf.errors.FailedPreconditionError as e:
        print(e)

Attempting to use uninitialized value v
	 [[Node: _retval_v_0_0 = _Retval[T=DT_FLOAT, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](v)]]


TensorFlow provides a function, `tf.global_variables_initializer`, which returns an op that can be evaluated to do just that:

In [8]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    print('The value for v is {}'.format(session.run(v)))

The value for v is [ 0.  0.]


A variable's value persists across a session unless it is updated by running an assignment op. For instance, the op returned by `tf.assign_add` can be used to increment a variable's value:

In [9]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    # The value for v persists across session.run calls...
    print('The value for v is {}'.format(session.run(v)))
    print('The value for v is {}'.format(session.run(v)))
    # ... until it is updated by running an assignment op.
    session.run(v.assign_add([1, 2]))
    print('The value for v is {}'.format(session.run(v)))

The value for v is [ 0.  0.]
The value for v is [ 0.  0.]
The value for v is [ 1.  2.]


In addition to `tf.assign_add`, the `tf.assign_sub` and `tf.assign` functions return ops which decrement and assign a variable's value, respectively.

## Placeholder

Oftentimes the computation we define depends on data which we don't yet have. For instance, the output of a neural network depends on a user-defined input which is only specified at runtime.

_Placeholder_ tensors are used to represent this data. They can be created via `tf.placeholder`:

In [10]:
p = tf.placeholder(dtype=tf.float32, shape=[], name='p')
print(p)

Tensor("p:0", shape=(), dtype=float32)


Once again, the `name` argument is optional, but it is good practice to provide it.

Because it has no pre-defined value, evaluating a placeholder tensor raises an error:

In [11]:
with tf.Session() as session:
    try:
        session.run(p)
    except tf.errors.InvalidArgumentError as e:
        # Cutting through the error message...
        print('\n'.join(str(e).split('\n')[-3:]))

InvalidArgumentError (see above for traceback): You must feed a value for placeholder tensor 'p' with dtype float
	 [[Node: p = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]



Its value must be _explicitly_ passed to `session.run` via the `feed_dict` argument, which expects a `dict` mapping tensors to their value:

In [12]:
with tf.Session() as session:
    # feed p the value 42
    print('The value for p is {}'.format(session.run(p, feed_dict={p: 42})))
    # feed p the value 21
    print('The value for p is {}'.format(session.run(p, feed_dict={p: 21})))

The value for p is 42.0
The value for p is 21.0


## Random

Another useful input tensor to have in our toolbox is the random tensor. The random seed can be set globally via `tf.set_random_seed`:

In [13]:
tf.set_random_seed(1234)

There are many random distributions to choose from in TensorFlow. Let's look at `tf.random_uniform`:

In [14]:
r = tf.random_uniform(
    shape=[], minval=0.0, maxval=1.0, dtype=tf.float32, name='r')

A random tensor's value changes randomly between `session.run` calls, but the sequence of those random values stays the same across different sessions:

In [15]:
with tf.Session() as session:
    print('The value for r is {}'.format(session.run(r)))
    print('The value for r is {}'.format(session.run(r)))

with tf.Session() as session:
    print('The value for r is {}'.format(session.run(r)))
    print('The value for r is {}'.format(session.run(r)))

The value for r is 0.8478444814682007
The value for r is 0.23446130752563477
The value for r is 0.8478444814682007
The value for r is 0.23446130752563477


# Combining tensors

Tensors can be combined in various ways using what TensorFlow calls operations, or _ops_. Ops can take zero or more tensors as input and produce zero or more tensors as output, with or without side effects.

We have already dealt with ops when initializing or assigning values to variables, but there are _many_ more TensorFlow functions which can be used to create ops.

The best way to discover new useful ops is to browse the [TensorFlow Python API](https://www.tensorflow.org/api_docs/python/). For instance, we can discover that there exists a function, `tf.add`, which adds two tensors together and returns a tensor representing the output:

In [16]:
one_plus_three = tf.add(1, 3)

with tf.Session() as session:
    print('1 + 3 = {}'.format(session.run(one_plus_three)))

1 + 3 = 4


Note that TensorFlow also offers syntactic sugar by overriding some Python operators like `+`, `-`, `*`, and `/`:

In [17]:
one_plus_four = tf.constant(1) + tf.constant(4)

with tf.Session() as session:
    print('1 + 4 = {}'.format(session.run(one_plus_four)))

1 + 4 = 5


# Where tensors live

So far we created tensors and combined them together, but we have not explicitly dealt with the computation graph itself. Which computation graph are these tensors part of, then?

By default, TensorFlow stores all tensors and operations in a **default graph**, which can be accessed as follows:

In [18]:
# Access the default graph
default_graph = tf.get_default_graph()
# Print first four operations defined in the graph
print(default_graph.get_operations()[:4])

[<tf.Operation 'c' type=Const>, <tf.Operation 'v/Initializer/zeros' type=Const>, <tf.Operation 'v' type=VariableV2>, <tf.Operation 'v/Assign' type=Assign>]


We can change that default graph to be another graph:

In [19]:
# Create a new Graph
alternate_graph = tf.Graph()

# Print the number of operations defined in the default and alternate graphs
print('default graph contains {} operations, '.format(len(default_graph.get_operations())) +
      'alternate graph contains {} operations'.format(len(alternate_graph.get_operations())))

# Use the alternate graph as the default graph
with alternate_graph.as_default():
    tf.constant(0, name='a')
    tf.constant(1, name='b')
    print(alternate_graph.get_operations())

# Print again the number of operations defined in the default and alternate graphs
print('default graph contains {} operations, '.format(len(default_graph.get_operations())) +
      'alternate graph contains {} operations'.format(len(alternate_graph.get_operations())))

default graph contains 23 operations, alternate graph contains 0 operations
[<tf.Operation 'a' type=Const>, <tf.Operation 'b' type=Const>]
default graph contains 23 operations, alternate graph contains 2 operations


In practice, you usually won't have to deal with multiple computation graphs. We will however use multiple computation graphs in this tutorial to isolate ops in their own namespace where appropriate.

# Gradients

TensorFlow supports _automatic differentiation_, i.e., it can compute the derivative of scalars with respect to tensors in the graph and represent the result as a symbolic expression.

Take for instance the linear equation

In [20]:
x = tf.placeholder(dtype=tf.float32, shape=[], name='p')
y = 3 * x + 2

We can compute the derivative of `y` with respect to `x` with the `tf.gradients` function:

In [21]:
dy_dx, = tf.gradients(ys=y, xs=[x])

We can verify that the gradient evaluates to 3 as expected:

In [22]:
with tf.Session() as session:
    dy_dx_val = session.run(dy_dx)
    print('The gradient of y with respect '
          'to x is {}'.format(dy_dx_val))

The gradient of y with respect to x is 3.0


_Note: Some of you may have noticed that TensorFlow did not complain despite no value being provided for the `x` placeholder. This is because even though `x` is part of the computation graph, the derivative of `y` with respect to `x` does not involve `x`, and therefore evaluating it does not require a value to be passed for `x`._

# Exercise

Find the minimum of the expression

$$2(x - 2)^2 + 2(y + 3)^2$$

using gradient descent by filling in the following code block:

In [23]:
# Avoid polluting the default graph by using an alternate graph
with tf.Graph().as_default():
    tf.set_random_seed(1234)

    # Create two scalar variables, x and y, initialized at random.
    # x = WRITEME.
    # y = WRITEME.

    # Create a tensor z whose value represents the expression
    #     2(x - 2)^2 + 2(y + 3)^2
    # z = WRITEME.
    
    # Compute the gradients of z with respect to x and y.
    # dx, dy = WRITEME.
    
    # Create an assignment expression for x using the update rule
    #    x <- x - 0.1 * dz/dx
    # and do the same for y.
    # x_update = WRITEME.
    # y_update = WRITEME.
    
    with tf.Session() as session:
        # Run the global initializer op for x and y.
        # WRITEME.
        
        for _ in range(10):
            pass
            # Run the update ops for x and y.
            # WRITEME.
            
            # Retrieve the values for x, y, and z, and print them.
            # x_val, y_val, z_val = WRITEME.
            # print('x = {:4.2f}, y = {:4.2f}, z = {:4.2f}'.format(x_val, y_val, z_val))

## Solution

In [ ]:
%load tensorflow_exercise_solution.py

# Optimization made easy

The solution to the exercise above can be shortened quite a bit by taking advantage of TensorFlow's optimization features. Here is the graph we were working with:

In [25]:
tf.set_random_seed(1234)
x = tf.get_variable(name='x', shape=[], dtype=tf.float32,
                    initializer=tf.random_normal_initializer())
y = tf.get_variable(name='y', shape=[], dtype=tf.float32,
                    initializer=tf.random_normal_initializer())

z = 2 * (x - 2) ** 2 + 2 * (y + 3) ** 2

TensorFlow provides utility classes to facilitate optimization in computation graphs. These classes inherit from `tf.train.Optimizer`. Let's look at the simplest one, `tf.train.GradientDescentOptimizer`.

We instantiate the `tf.train.GradientOptimizer` by passing it a scalar learning rate. Note that the learning rate itself can be symbolic, and is allowed to vary across a session.

In [26]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

We can then call the optimizer's `minimize` function to obtain an op with, when evaluated, does a gradient descent step on the variables we specified:

In [27]:
update_op = optimizer.minimize(loss=z, var_list=tf.trainable_variables())

Here we took advantage of the fact that all variables created via `tf.get_variable` can be accessed as a list using the `tf.trainable_variables` function.

(Note: you can pass `trainable=False` to `tf.get_variable` to exclude a certain variable from ending up in that list.)

The code then proceeds as before:

In [28]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    for _ in range(10):
        session.run(update_op)
        x_val, y_val, z_val = session.run([x, y, z])
        print('x = {:4.2f}, y = {:4.2f}, z = {:4.2f}'.format(x_val, y_val, z_val))

x = 0.52, y = -1.75, z = 7.49
x = 1.11, y = -2.25, z = 2.70
x = 1.47, y = -2.55, z = 0.97
x = 1.68, y = -2.73, z = 0.35
x = 1.81, y = -2.84, z = 0.13
x = 1.89, y = -2.90, z = 0.05
x = 1.93, y = -2.94, z = 0.02
x = 1.96, y = -2.97, z = 0.01
x = 1.98, y = -2.98, z = 0.00
x = 1.99, y = -2.99, z = 0.00


# Control flow

TensorFlow implements symbolic operations for executing common control flow structures, like if-statements and while loops.

This is usually where people get confused: despite appearances, there is a *big* difference between a regular control flow statement and its symbolic counterpart. **When working with control flow ops, you should always keep in mind that the code you write _defines the computation graph_, it does not _execute_ that computation.**

## If-statement

To illustrate this, let's look at a regular if-statement:

In [29]:
regular_if_graph = tf.Graph()
with regular_if_graph.as_default():
    # Define computation graph
    x = 3
    y = tf.placeholder(dtype=tf.float32, shape=[], name='y')
    if x < 4:
        z = y + 1
    else:
        z = y - 1
    
    # Run the computation graph on some input
    with tf.Session() as session:
        print(session.run(z, feed_dict={y: 0}))


1.0


This piece of code is fairly simple: depending on the value of `x`, we either add 1 to `y` or subtract 1 from it. However, it depends on the fact that *the value of `x` was known when the graph was created*. In fact, because the `else` statement never gets executed, **the expression `y - 1` doesn't even appear in the computation graph**:

In [30]:
print(regular_if_graph.get_operations())

[<tf.Operation 'y' type=Placeholder>, <tf.Operation 'add/y' type=Const>, <tf.Operation 'add' type=Add>]


Clearly, this approach does not work when the value of `x` is not known in advance. Try the same code with a placeholder `x` and you will be greeted with an error message:

In [31]:
with tf.Graph().as_default():
    x = tf.placeholder(dtype=tf.float32, shape=[], name='x')
    y = tf.placeholder(dtype=tf.float32, shape=[], name='y')
    try:
        if x < 4:
            z = y + 1
        else:
            z = y - 1
    except TypeError as e:
        print(str(e).split('\n')[0])

Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.


For this use case, TensorFlow implements function called `tf.cond` which acts as a symbolic counterpart to the if-statement:

In [32]:
symbolic_if_graph = tf.Graph()
with symbolic_if_graph.as_default():
    # Define computation graph
    x = tf.placeholder(dtype=tf.float32, shape=[], name='x')
    y = tf.placeholder(dtype=tf.float32, shape=[], name='y')
    z = tf.cond(
        pred=x < 4,
        true_fn=lambda: y + 1,
        false_fn=lambda: y - 1)
    
    # Run the computation graph on some inputs
    with tf.Session() as session:
        print(session.run(z, feed_dict={x: 3, y: 0}))
        print(session.run(z, feed_dict={x: 5, y: 0}))

1.0
-1.0


The `tf.cond` function takes a predicate `pred`, a subgraph-creating function `true_fn`, and a subgraph-creating function `false_fn` as input. The predicate is a *symbolic* boolean which is used to decide which branch of the conditional is executed. The two graph-creating functions take no argument as input, create a computation subgraph, and return its symbolic output.

Looking at the operations defined in the graph above reveals a very different picture:

In [33]:
print(symbolic_if_graph.get_operations())

[<tf.Operation 'x' type=Placeholder>, <tf.Operation 'y' type=Placeholder>, <tf.Operation 'Less/y' type=Const>, <tf.Operation 'Less' type=Less>, <tf.Operation 'cond/Switch' type=Switch>, <tf.Operation 'cond/switch_t' type=Identity>, <tf.Operation 'cond/switch_f' type=Identity>, <tf.Operation 'cond/pred_id' type=Identity>, <tf.Operation 'cond/add/y' type=Const>, <tf.Operation 'cond/add/Switch' type=Switch>, <tf.Operation 'cond/add' type=Add>, <tf.Operation 'cond/sub/y' type=Const>, <tf.Operation 'cond/sub/Switch' type=Switch>, <tf.Operation 'cond/sub' type=Sub>, <tf.Operation 'cond/Merge' type=Merge>]


**The takeaway here is that because TensorFlow separates graph definition from graph execution, we have to adjust our mental model of what happens behind the scenes, even for a seemingly innocuous if-statement.**

Now that this warning is out of the way, we can look at some other symbolic control-flow functions TensorFlow implements.

## Case

The `tf.case` function implements a symbolic counterpart to the `case` control flow statement. It takes a sequence of (predicate, subgraph-creating function) tuples as input:

In [34]:
with tf.Graph().as_default():
    # Define computation graph
    x = tf.placeholder(dtype=tf.int32, shape=[], name='x')
    y = tf.placeholder(dtype=tf.float32, shape=[], name='y')
    z = tf.case(
        pred_fn_pairs=[(tf.equal(x, 0), lambda: y + 1),
                       (tf.equal(x, 1), lambda: y - 1),
                       (tf.equal(x, 2), lambda: y * 2)],
        default=lambda: y / 2)
    
    # Run the computation graph on some inputs
    with tf.Session() as session:
        print(session.run(z, feed_dict={x: 0, y: 2}))
        print(session.run(z, feed_dict={x: 1, y: 2}))
        print(session.run(z, feed_dict={x: 2, y: 2}))
        print(session.run(z, feed_dict={x: 3, y: 2}))

3.0
1.0
4.0
1.0


## While loop

The `tf.while_loop` function implements a symbolic counterpart to the `while` control flow statement. It takes a `cond` subgraph-creating function, a `body` subgraph-creating function and a `loop_vars` sequence of tensors.

You can think of `loop_vars` as the initial state of all tensors which change from one iteration of the loop to the other. The `cond` and `body` functions takes a sequence of tensors with the same length as `loop_vars` as input; you can think of them as the current state the `loop_vars`. The `cond` function returns a symbolic boolean telling whether the loop should be executed or not. The `body` function returns a sequence of tensors representing the new values the `loop_vars`.

Here is an example of a sequential implementation of the Fibonacci sequence using `tf.while_loop`:

In [35]:
with tf.Graph().as_default():
    # Define computation graph
    n = tf.placeholder(tf.int32, shape=[], name='n')
    i = tf.constant(2)
    a = tf.constant(0)
    b = tf.constant(1)

    _, _, nth_fib = tf.while_loop(
        cond=lambda i, a, b: i < n,
        body=lambda i, a, b: (i + 1, b, a + b),
        loop_vars=(i, a, b))

    # Run the computation graph on some inputs
    with tf.Session() as session:
        print(session.run(nth_fib, feed_dict={n: 6}))
        print(session.run(nth_fib, feed_dict={n: 8}))

5
13


We define a placeholder tensor `n` representing which element of the Fibonacci sequence to compute. We then instantiate three constant tensors that act as the `loop_vars`: a counter `i` and two values `a` and `b` representing the elements $i - 1$ and $i$ of the Fibonacci sequence, respectively.

The `cond` argument passed to `tf.while_loop` indicates that the body should execute as long as $i < n$. The `body` argument itself increments `i` by one and updates the value for `a` and `b` so they reflect elements $i$ and $i + 1$ of the Fibonacci sequence, respectively (reminder: $fib(i + 1) = fib(i) + fib(i - 1)$).

## Scan

Another useful function TensorFlow implements is `tf.scan`, which "scans" over the elements of its input and applies some possibly stateful function to them.

Here is an example of a cumulative sum implemented using `tf.scan`:

In [36]:
with tf.Graph().as_default():
    # Define computation graph
    x = tf.placeholder(tf.int32, shape=[None], name='x')

    c = tf.scan(
        fn=lambda a, x_t: a + x_t,
        elems=x,
        initializer=tf.constant(0))
    
    # Run the computation graph on some input
    with tf.Session() as session:
        print(session.run(c, feed_dict={x: [1, 2, 3, 4]}))

[ 1  3  6 10]


We pass `tf.scan` three arguments as inputs: a subgraph-creating function `fn`, a tensor (or sequence of tensors) `elems` to loop over, and a tensor (or list of tensors) `initializer` of initial accumulator values.

The scan function loops over the first axis of `elems` (or, if it's a sequence, over the first axes of every element in the sequence). The `fn` function takes the accumulator value and the current loop element as input, and returns a new value for the accumulator.

The `tf.scan` function itself returns a sequence of accumulated values.

# Scaling up to large computation graphs

We have now covered the bare minimum that would allow you to do machine learning with TensorFlow. We have not covered _every_ TensorFlow op, but you now possess the knowledge required to browse through the [TensorFlow Python API](https://www.tensorflow.org/api_docs/python/) and find what you need.

We will now concentrate on ways to scale what you learned to actual machine learning problems without increasing the maintenance complexity too much.

## Variable and name scopes

TensorFlow uses a soft convention for op and variable names: an op or variable that is part of a hierarchy should have a name that conveys its location in the hierarchy, with the `'/'` character being used to separate different levels in the hierarchy. For instance, a good name for the bias vector of the second layer of the model would be `'model/layer2/b'`.

In order to reduce code duplication and facilitate maintenance, TensorFlow provides two context managers, named `tf.name_scope` and `tf.variable_scope`, inside which variables and ops that are created see their name prepended with the name of the enclosing scope. The difference between the two is that `tf.variable_scope` operates on _all_ names, whereas `tf.name_scope` operates on all _but_ variable names:

In [37]:
# Variable scopes operate on all tensors
with tf.variable_scope('foo'):
    # Scopes can be nested
    with tf.variable_scope('bar'):
        print(tf.get_variable('a', shape=[]).name)
        print(tf.constant(0.0, name='b').name)
# Name scopes do not operate on variables
with tf.name_scope('machine'):
    with tf.name_scope('learning'):
        print(tf.get_variable('a', shape=[]).name)
        print(tf.constant(0.0, name='b').name)

foo/bar/a:0
foo/bar/b:0
a:0
machine/learning/b:0


## Device placement

In this tutorial we have not bothered with the specific placement (CPU or GPU) of our ops, i.e., the device on which they are executed, mostly because the examples we considered were so small in scale that GPU acceleration makes little to no sense.

However, in general, we would like most of our large operations (such as matrix-matrix multiplications) to take place on the GPU. Fortunately for us, TensorFlow already handles device placement for us behind the scenes. The short story is that TensorFlow will try to place all possible ops on all available GPUs, which is a good default to have.

Because we are sharing workstations with multiple GPUs, this means we need to be careful in allowing TensorFlow to see only the GPUs we want to use. This is achieved by setting the `CUDA_VISIBLE_DEVICES` environment variable. For instance, to allow TensorFlow to see GPUs 0 and 2, set it to

```bash
CUDA_VISIBLE_DEVICES=0,2
```

To allow TensorFlow to see GPU 0 only, set it to

```bash
CUDA_VISIBLE_DEVICES=0
```

To disallow it to see any GPU, set it to

```bash
CUDA_VISIBLE_DEVICES=""
```

In addition, there may be situations in which we want fine-grained control over device placement. For instance, we may want on-the-fly input pre-processing to be computed on the CPU and reserve the GPU for inference. In that case, use the `tf.device` context manager:

```python
# All ops created within this context are forced to be placed on CPU
with tf.device('/cpu:0'):
    x = # some tensor
    preprocessed_x = # some pre-processing on x

# All ops created within this context are forced to be placed on GPU
with tf.device('/gpu:0'):
    y = # some mapping from x to y
```

You can find more information on manual device placement in the [TensorFlow documentation](https://www.tensorflow.org/tutorials/using_gpu#manual_device_placement).

# Graph visualization with TensorBoard

**TODO: WRITEME**